### 提供精确的属性预估值

In [10]:
import pandas as pd
import xgboost as xgb
import numpy as np
def predict(wob,T,pred_y):
    #数据预处理
    df = pd.DataFrame(columns=['wob_min','wob_max','wob_mean','wob_std','wob_skew','wob_kurt','T_min','T_max','T_mean','T_std','T_skew','T_kurt'])

    wob_min = np.min(wob)
    wob_max = np.max(wob)
    wob_mean = np.mean(wob)
    wob_std = np.std(wob, ddof=1)  # ddof=0 对于样本标准差，ddof=1 对于总体标准差
    wob_skew = pd.Series(wob).skew()
    wob_kurt = pd.Series(wob).kurt()

    T_min = np.min(T)
    T_max = np.max(T)
    T_mean = np.mean(T)
    T_std = np.std(T, ddof=1)  # ddof=0 对于样本标准差，ddof=1 对于总体标准差
    T_skew = pd.Series(T).skew()
    T_kurt = pd.Series(T).skew()

    row = {'wob_min':wob_min,'wob_max':wob_max,'wob_mean':wob_mean,'wob_std':wob_std,'wob_skew':wob_skew,'wob_kurt':wob_kurt,'T_min':T_min,'T_max':T_max,'T_mean':T_mean,'T_std':T_std,'T_skew':T_skew,'T_kurt':T_kurt}
    df.loc[len(df)] = row
    res = {}
    for pred in pred_y:
        # 加载已保存的模型
        loaded_bst = xgb.Booster()
        loaded_bst.load_model("./模型/model_"+pred+".xgb")  # 从model.xgb文件加载模型
        # 使用加载后的模型进行预测
        feature_name =df.columns
        dtest = xgb.DMatrix(df,feature_names=feature_name)
        bst_ypred = loaded_bst.predict(dtest)
        print(pred+':',bst_ypred)
        res[pred] = bst_ypred
    return res





#模型输入数据，3秒的wob和T数据 以及要预测的属性名称
#['设备S','设备Z','静态抗压强度', '弹性模量', '泊松比', '抗拉强度', '黏聚力', '内摩擦角', '回弹均值', '动态强度', '滑动摩擦系数', '声级', '波速', '密度均值', '渗透率', '孔隙度', '标定温度']

wob = [3.52979,3.5755,3.02758,3.65089,3.30212,3.63342,3.38165,3.46744,3.56689,3.00286]
T = [11.48,9.597,10.277,11.4,7.998,9.718,10.092,11.97,9.343,9.56]
pred_y = ['静态抗压强度', '弹性模量', '泊松比','抗拉强度', '黏聚力']


#调用模型
predict(wob,T,pred_y)

静态抗压强度: [111.04034]
弹性模量: [53.288494]
泊松比: [0.25958583]
抗拉强度: [6.71879]
黏聚力: [12.006973]


{'静态抗压强度': array([111.04034], dtype=float32),
 '弹性模量': array([53.288494], dtype=float32),
 '泊松比': array([0.25958583], dtype=float32),
 '抗拉强度': array([6.71879], dtype=float32),
 '黏聚力': array([12.006973], dtype=float32)}

### 提供精确的材料类型

In [8]:
import pandas as pd
import xgboost as xgb
import numpy as np
def predict_kind(wob,T):
    #数据预处理
    df = pd.DataFrame(columns=['wob_min','wob_max','wob_mean','wob_std','wob_skew','wob_kurt','T_min','T_max','T_mean','T_std','T_skew','T_kurt'])

    wob_min = np.min(wob)
    wob_max = np.max(wob)
    wob_mean = np.mean(wob)
    wob_std = np.std(wob, ddof=1)  # ddof=0 对于样本标准差，ddof=1 对于总体标准差
    wob_skew = pd.Series(wob).skew()
    wob_kurt = pd.Series(wob).kurt()
    T_min = np.min(T)
    T_max = np.max(T)
    T_mean = np.mean(T)
    T_std = np.std(T, ddof=1)  # ddof=0 对于样本标准差，ddof=1 对于总体标准差
    T_skew = pd.Series(T).skew()
    T_kurt = pd.Series(T).skew()
    row = {'wob_min':wob_min,'wob_max':wob_max,'wob_mean':wob_mean,'wob_std':wob_std,'wob_skew':wob_skew,'wob_kurt':wob_kurt,'T_min':T_min,'T_max':T_max,'T_mean':T_mean,'T_std':T_std,'T_skew':T_skew,'T_kurt':T_kurt}
    df.loc[len(df)] = row


    #计算各种属性，计算其与标注属性之间的距离
    cols = ['设备S', '静态抗压强度', '弹性模量', '泊松比', '抗拉强度', '黏聚力', '内摩擦角', '波速', '密度均值', '孔隙度', '标定温度']
    res_list = []
    for pred in cols:
        # 加载已保存的模型
        loaded_bst = xgb.Booster()
        loaded_bst.load_model("./模型/model_"+pred+".xgb")  # 从model.xgb文件加载模型
        # 使用加载后的模型进行预测
        feature_name =df.columns
        dtest = xgb.DMatrix(df,feature_names=feature_name)
        bst_ypred = loaded_bst.predict(dtest)
        res_list.append(bst_ypred[0])

    cl2 = [101.4,110.95,53.58,0.26,6.65,11.85,38.44,5.49,2.86,2.08,60.21]
    cl4 = [71.5,117.49,58.62,0.33,10.68,13.53,46.34,5.24,3.28,2.09,57.99]
    cl5 = [67.9,126.65,72.11,0.27,12.67,21.09,44.90,6.76,3.45,2.06,63.23]
    cl7 = [65.7,53.53,31.54,0.28,4.56,8.02,45.61,5.95,3.67,14.49,57.11]
    cl8 = [72.9,164.87,53.87,0.23,5.76,12.01,44.17,4.72,4.32,1.87,63.54]
    cl10 = [69.3,48.66,47.03,0.24,4.96,18.04,44.86,5.31,3.87,2.97,57.14]
    cl11 = [75.2,192.15,77.85,0.21,7.74,17.81,45.53,6.58,3.06,1.14,65.07]
    cl13 = [70.8,129.06,85.21,0.26,6.84,21.22,38.96,6.41,3.81,1.25,56.53]
    cl55 = [68.7,39.96,29.81,0.24,3.11,16.99,31.80,2.97,2.74,18.47,53.31]
    cl21 = [64.5,61.84,32.55,0.27,4.72,13.75,41.61,3.43,2.64,11.68,55.67]
    cl22 = [78.1,131.05,55.01,0.19,7.45,20.80,46.75,5.10,2.53,1.16,66.72]

    resss = {'DTj':cl2,'Fec-BR':cl4,'DT':cl5,'Py':cl7,'Feb-BR':cl8,'BR':cl10,'CF':cl11,'Fed-DT':cl13,'ST':cl55,'GS':cl21,'GG':cl22}
    min_dis = float('inf')
    cl = ''
    for key,temp in resss.items():

        dis = np.linalg.norm(np.array(res_list) - np.array(temp))

        if dis < min_dis:
            min_dis = dis
            cl = key
    return cl


#模型输入数据  3秒的wob和T数据
wob = [3.52979,3.5755,3.02758,3.65089,3.30212,3.63342,3.38165,3.46744,3.56689,3.00286]
T = [11.48,9.597,10.277,11.4,7.998,9.718,10.092,11.97,9.343,9.56]
predict_kind(wob,T)

'DTj'